# Scraping Intro Homework: Wikipedia Table

In this assignment, we'll be extracting data from Wikipedia's table of the tallest buildings in Brooklyn: https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Brooklyn

### 0) Setup

Import `requests`, `BeautifulSoup`, and `pandas`. Although this homework uses `BeautifulSoup`, you can choose to use `lxml` instead, if you prefer.

In [3]:
import requests
import lxml.html
import pandas as pd
import pprint as pp

### 1) Grab the HTML for the webpage linked above

Use `requests` to get the HTML, assigning it to a variable

In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Brooklyn"
page = requests.get(URL).text

### 2) Convert the HTML into a `BeautifulSoup` object

In [5]:
bk_DOM = lxml.html.fromstring(page)

# pp.pprint(bk_DOM.text_content())

### 3) Use `.select(...)` (and `[0]`) to select the main table

That's the one directly under the "Tallest buildings" heading.

Print out the first 100 characters of text from the table to make sure you have the right one.

In [6]:
table = bk_DOM.cssselect("table")[0]

# print(table.text_content())

### 4) Use `.select(...)` (and `[0]`) again to select the table's first row

... which is its header. (Reminder that the `<thead>` tag is optional. Wikipedia tables don't use it.)

In [7]:
header = table.cssselect("tr")[0]
col_els = header.getchildren()

### 5) Extract the column names from that header

Use `.strip()` to remove any leading or trailing whitespace from the names.

First, try doing this with a standard `for` loop:

In [8]:
cols = list(map(lambda x: x.text.strip(), col_els))

Now try to do it with a list comprehension:

In [9]:
cols_list = [ col.text.strip() for col in col_els ]

### 6) Select all non-header row *elements* from the table

Since the header was the first row, you'll want to skip that one. How many rows are there? (Check with your eyes that this number matches what you deduce from the rankings column in the browser-rendered table.)

In [10]:
table_body = table.cssselect("tr")[1:]

### 7) For each row, extract the text of each cell into a Python list

First, try this as two nested `for` loops:

In [11]:
bk_table = []

# nested list aka list of lists
for row in table_body:
    cells = []
    cell_els = row.getchildren()

    for cell in cell_els:
        cells.append(cell.text_content().strip())

    bk_table.append(cells)

# pp.pprint(bk_table[:5])

Now try with two list comprehensions, one nested in the other:

In [12]:
# nested list
bk_table_list = [[cell.text_content().strip() for cell in row] for row in table_body]

# pp.pprint(list_table[:5])

### 8) Turn the data you've extracted into a `pandas` `DataFrame`

In [13]:
df = pd.DataFrame(bk_table, columns=cols)
bk_df = df.copy()
bk_df.head()

,Rank,Name,Image,Height,Floors,Year completed,Notes
0,1,The Brooklyn Tower,,"1,073 (327)",93,2022,Topped out in October 2021.[2][22][23][24]
1,2,Brooklyn Point,,720 (219),68,2019,The final phase of Extell's City Point develop...
2,3,11 Hoyt,,626 (191),51,2020,Topped out in June 2019.[31] A redevelopment o...
3,4,The Hub,,611 (186),52,2017,Also known as 333 Schermerhorn Street. Topped ...
4,5,AVA DoBro,,596 (182),58,2015,"Also known as 100 Willoughby Street, Avalon Wi..."


### 9) Which years are represented by at least 5 buildings?

In [14]:
#https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
#check
df['Year completed'].value_counts()

bk_df['Year completed'].value_counts().loc[lambda x: x >= 5].reset_index()['Year completed']

0    2022
1    2020
2    2016
Name: Year completed, dtype: object

In [18]:
# alternatively!!!

year_counts = df['Year completed'].value_counts()
year_counts[year_counts >= 5]

Year completed
2022    11
2020     6
2016     6
Name: count, dtype: int64

### 10) How many total floors do all the buildings have, combined?

In [15]:
floors = sum(bk_df['Floors'].astype(int))
print("Total floors on all buildings are:", floors)

Total floors on all buildings are: 2603


### 11) How many of the buildings have their own Wikipedia page?

For this, you'll need to query the row elements again; the information won't have been extracted into your `DataFrame`. (Hint: Whether a building has its own Wikipedia page isn't an explicit piece of data, but something you can infer from the presence of a particular sub-element.)

In [16]:
# get rows
# read for a-tag link in name
# write new element to list
# ---> if a-tag link exists = 1
# ---> no a tag = 0
# write to df --> do df['link'].value_counts()

In [26]:
# jeremy's answer:
## select the row, a tag count.
second_cells = [ row.cssselect("td")[1] for row in table_body ]

sum(len(cell.cssselect("a")) > 0 for cell in second_cells)

17

### 12) How many have an image?

You could do this by testing for the presence of another element:

Or through information that's already in your `DataFrame`:

In [28]:
# check if image col is not empty or != None?
# count how many aren't empty.

(bk_df["Image"] != "—").sum()

56

### Bonus challenge

If we tried to run the same code on https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_New_York_City instead, the results wouldn't be quite right. Try it. Then, examining the HTML of that page, try to figure out why.

If you want an extra-extra challenge, try writing code that would parse that table correctly.

---

---

---